In [1]:
import pandas as pd
import csv
import math
import random
import string

In [2]:
sampleSubmission = pd.read_csv("../../data/SampleSubmission.csv")
seasons = pd.read_csv('../../data/Seasons.csv')
seeds = pd.read_csv('../../data/TourneySeeds.csv')
slots = pd.read_csv('../../data/TourneySlots.csv')
tourneyData = pd.read_csv('../../data/TourneyDetailedResults.csv')
seasonData = pd.read_csv('../../data/RegularSeasonDetailedResults.csv')
teams = pd.read_csv('../../data/Teams.csv')
kenPom = pd.read_csv("../../data/KenPom.csv")
teamSpelling = pd.read_csv("../../data/TeamSpellings.csv")

In [3]:
# Concatenate 'teams' and 'teamSpelling' into one complete df of team names
teams = teams[['Team_Name', 'Team_Id']]
teamSpelling.columns = ['Team_Name', 'Team_Id']
teams = pd.concat([teams, teamSpelling])

In [4]:
# Get numerical seeds for each team
seeds['Seed'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
allSeeds = dict()
for index, row in seeds.iterrows():
    if row['Season'] not in allSeeds.keys():
        allSeeds[row['Season']] = dict()
    allSeeds[row['Season']][row['Team']] = row['Seed']

In [218]:
teamIDs = list()
drops = list()
wr = list()
lTeams = pd.DataFrame(teams['Team_Name'].str.lower().str.strip("."), columns=['Team_Name'])
lTeamIDs = pd.DataFrame(teams['Team_Id'], columns = ['Team_Id'])
lTeams = pd.concat([lTeams, lTeamIDs], axis=1)
exclude = set(string.punctuation)
for index, row in kenPom.iterrows():
    lTeam = ''.join(ch for ch in row['Team'] if ch not in exclude)
    lTeam = lTeam.lower()
    lTeam = lTeam.rstrip(" ")
    team = lTeams.loc[lTeams['Team_Name'] == lTeam]
    if team.shape[0] != 0:
        teamIDs.append(team['Team_Id'].iloc[0])
    else:
        drops.append(index)
        wr.append(lTeam)
kenPom = kenPom.drop(drops)

In [219]:
kenPom = kenPom.reset_index(drop=True)
kenPom = pd.concat([kenPom, pd.Series(teamIDs, name="Team_Id")], axis=1)

In [234]:
drops = list()
for index, row in kenPom.iterrows():
    if pd.isnull(row['Seed']):
        if row['Team_Id'] in allSeeds[row['Year']]:
            kenPom.set_value(index, 'Seed', allSeeds[row['Year']][row['Team_Id']])
        else:
            drops.append(index)
kenPom.drop(drops)

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank,Team_Id
0,2002,1,Duke,ACC,31,4,1,0.9689,117.7,1,...,223,0.7273,13,106.7,11,98.0,32,0.6398,47,1181
1,2002,2,Maryland,ACC,32,4,1,0.9551,117.5,2,...,104,0.7214,17,106.8,10,98.3,38,0.5467,112,1268
2,2002,3,Cincinnati,CUSA,31,4,1,0.9508,115.8,6,...,165,0.6457,56,103.7,75,98.4,42,0.5769,84,1153
3,2002,4,Kansas,B12,31,4,1,0.9482,116.0,5,...,109,0.7419,10,107.8,4,98.4,39,0.6842,26,1242
4,2002,5,Oklahoma,B12,31,5,2,0.9352,112.7,17,...,67,0.6903,30,106.4,15,99.2,68,0.4970,159,1328
5,2002,6,Indiana,B10,25,12,5,0.9251,111.4,23,...,269,0.8049,1,108.3,2,95.7,2,0.7943,5,1231
6,2002,7,Florida,SEC,22,9,5,0.9143,111.9,22,...,299,0.6938,25,105.2,37,98.0,33,0.4715,188,1196
7,2002,8,Illinois,B10,26,9,4,0.9083,112.7,19,...,120,0.7385,11,107.2,9,98.0,29,0.6606,38,1228
8,2002,9,Pittsburgh,BE,29,6,3,0.8984,110.3,35,...,171,0.5578,112,101.3,168,99.3,70,0.2337,326,1338
9,2002,10,Marquette,CUSA,26,7,5,0.8928,110.2,37,...,153,0.5946,90,102.8,105,99.5,76,0.4010,251,1266
